In [1]:
import json
import pickle
import random
from collections import defaultdict
from datetime import datetime
from itertools import chain
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold
from statsmodels.formula import api as formula_api

pandas.set_option('display.max_columns', 500)

Using TensorFlow backend.


In [2]:
# Define cross-validation and evaluation strategies.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

In [3]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        for hero_key in ('level', 'color', 'star'):
            result[f'''{hero_key}_{hero['id']}'''] += sign * hero[hero_key]


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'win': battle['win'], **result}

In [4]:
# Load the logs into a data frame.

def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "symmetrical" battle.
    """
    return series == False if series.name == 'win' else -series

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
# battles = pandas.concat((battles, battles.apply(invert_column)))  # add inverted battles
# battles = battles.sample(frac=1)  # shuffle
battles.head()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9,win
0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [5]:
battles.describe()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9
count,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.00000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.00000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000
mean,-0.779221,3.246753,0.848485,-1.017316,3.675325,-0.264069,-0.480519,0.051948,-0.264069,0.424242,-0.064935,2.112554,-1.679654,-0.099567,-0.138528,1.471861,-0.398268,-0.233766,-0.134199,-0.129870,-0.116883,-0.030303,0.030303,-0.225108,-0.064935,-0.194805,-0.103896,-0.142857,0.034632,-0.194805,-3.225108,-0.064935,-0.060606,-0.831169,0.225108,-1.103896,-0.242424,-6.051948,26.995671,6.917749,-6.575758,28.718615,-2.047619,-4.259740,0.467532,-2.168831,4.523810,-0.532468,14.969697,-13.987013,-0.809524,-1.116883,11.896104,-3.194805,-1.82684,-1.142857,-0.978355,-0.917749,-0.220779,0.238095,-1.878788,-0.675325,-1.480519,-0.917749,-1.116883,0.238095,-1.636364,-25.839827,-0.567100,-0.528139,-6.56710,1.844156,-8.857143,-1.298701,-0.277056,1.826840,0.363636,-0.393939,1.515152,-0.099567,-0.108225,0.025974,-0.099567,0.233766,0.004329,1.056277,-0.601732,-0.043290,-0.043290,0.082251,-0.125541,-0.095238,-0.043290,-0.034632,-0.034632,-0.025974,0.008658,-0.095238,-0.038961,-0.077922,-0.047619,-0.047619,0.017316,-0.060606,-1.212121,-0.012987,-0.025974,-0.264069,0.471861,-0.415584,-0.121212
std,2.575588,3.656403,2.876968,3.609125,3.626382,1.550444,4.998114,0.557076,1.293616,4.156935,1.240662,3.563751,3.033361,1.092882,0.940784,3.842309,1.587121,1.334128,1.310227,0.982685,0.884269,0.655372,0.460566,1.176388,0.864835,1.198248,1.723881,1.165231,1.152922,1.111665,4.020758,0.697904,0.656577,2.476336,4.248936,2.820350,1.076274,20.119143,29.593698,23.400617,28.992685,28.154134,12.698592,40.226771,5.013680,10.416211,33.374399,9.631359,28.475221,24.927980,8.496961,7.610589,30.489592,12.733085,10.53505,11.155869,7.436365,6.972442,5.880602,3.618734,9.560927,7.927017,9.115601,14.561032,9.400704,9.405804,9.349135,32.067936,6.095868,5.664369,20.28093,32.935272,22.596103,8.184925,0.937778,1.974923,1.229094,1.795104,1.548768,0.668255,2.110321,

In [6]:
# Split the dataframe.

x = battles.drop(['win'], axis=1)
y = battles['win']

In [7]:
# Analyse the model.

# https://github.com/statsmodels/statsmodels/issues/3931#issuecomment-343810211
stats.chisqprob = stats.chi2.sf
print(f'Shape: {x.shape}. Rank: {numpy.linalg.matrix_rank(x.values)}.')
# TODO: formula_api.Logit(y, x).fit().summary()

Shape: (231, 111). Rank: 105.


In [15]:
# Train, adjust hyper-parameters and evaluate.

numpy.random.seed(42)

# classifier = LogisticRegression(max_iter=1000, class_weight='balanced')
# param_grid = {'C': numpy.logspace(-3, 2, num=100)}

classifier = RandomForestClassifier(class_weight='balanced', n_jobs=4, random_state=42)
param_grid = {
    'n_estimators': [50, 51, 52, 53, 54],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'max_features': [None, 'auto', 'log2'],
}

grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=1).fit(x, y)
best_estimator = grid_search.best_estimator_

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {best_estimator.classes_}')

scores = cross_val_score(best_estimator, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.8138528138528138
Best params: {'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 52}
Classes: [False  True]
CV score: 0.8135491828553809 (std: 0.046471622861588034)


In [16]:
# Re-train the best model on the entire dataset.

best_estimator.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=52, n_jobs=4,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [17]:
# Save the model.

print(f'''
"""
Arena battle prediction model.
Auto-generated on {datetime.now().replace(microsecond=0)}.
X shape: {x.shape}.
Score: {scores.mean():.4f} (std: {scores.std():.4f}).
"""

import pickle

from {best_estimator.__class__.__module__} import {best_estimator.__class__.__qualname__}

feature_names = {list(x.columns)}
model: {best_estimator.__class__.__qualname__} = pickle.loads({pickle.dumps(best_estimator)})
'''.strip(), file=open('model.py', 'wt'))

In [18]:
# Display the feature importances.

pandas.DataFrame({'Feature': x.columns, 'Importance': best_estimator.feature_importances_}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=10)

,Importance
Feature,
level_7,0.059023
level_2,0.047305
color_7,0.045754
level_10,0.044973
level_15,0.042282
level_4,0.036449
color_15,0.034615
color_2,0.034515
star_7,0.026745


In [19]:
# Display prediction examples.

result = pandas.concat((
    pandas.Series(best_estimator.predict(x), index=battles.index, name='win_predicted'),
    pandas.Series(best_estimator.predict_proba(x)[:, 1], index=battles.index, name='win_probability'),
    battles,
), axis=1)
result['win_probability'] = result['win_probability'].apply('{:.2f}'.format)
result.head()

,win_predicted,win_probability,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9,win
0,False,0.08,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,False,0.00,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,False,0.08,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,False,0.04,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,False,0.10,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [13]:
'''

# Experimental NN model.
# I want it to catch an interaction between different heroes (e.g. Йорген is good in conjunction with Исмаил).

def create_model() -> keras.Model:
    model = Sequential()
    model.add(Dense(1000, input_dim=x.columns.size, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, input_dim=x.columns.size, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

numpy.random.seed(42)
scores = cross_val_score(keras_classifier, x, y, cv=cv)

print(f'Score: {scores.mean()} (std: {scores.std()})')

'''

None